# The role of air pollution on the propagation and mortality of COVID-19

Since the beginning of the COVID-19 pandemic, countless studies have been performed to gain a better insight about this virus. It has been for example known that the novel coronavirus is [able to spread through droplets](https://www.frontiersin.org/articles/10.3389/fpubh.2020.00163/full?fbclid=IwAR2rhRUdRpqZ-OGRId78hKvIXT36WDTUQlpR-JRH_0C-2ElafKv62QCrrwI), however [further research](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7444490/) suggests it might also spread through air pollutants, such as in PM2.5, PM10 and NO2. 
This analysis aims to confirm whether there is a correlation between air pollution levels and COVID-19 spread and lethality.


In [1]:
# Importing necessary libraries
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from zipfile import ZipFile
import io
import urllib3
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import panel as pn
from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets
pn.extension('plotly')
import researchpy as rp
import scipy.stats as stats
import numpy as np
import pylab

In [2]:
# Disables the warning when downloading
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


def zipped_downloader(url):
    """
    Downloads the files containing pollution data of the USA,
    unzips and creates a pandas dataframe.
    """
    response = requests.get(url, auth=HTTPBasicAuth('user', 'pass'),
                            stream=True, verify=False)
    # Unzips the file
    with ZipFile(io.BytesIO(response.content)) as myzip:
        # Open first file in folder
        with myzip.open(myzip.namelist()[0]) as myfile:
            # Reads csv and creates a pandas dataframe
            df = pd.read_csv(myfile)
            return df


def massDownloader(url_key, zipped=False):
    urls = {'2020':'https://aqs.epa.gov/aqsweb/airdata/daily_88101_2020.zip',
            '2021':'https://aqs.epa.gov/aqsweb/airdata/daily_88101_2021.zip',
            'covid':'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv'}
    if zipped == True:
        df = zipped_downloader(urls[url_key])
    else:
        df = pd.read_csv(urls[url_key])
    return df


covid19_usa = massDownloader('covid', zipped=False)
PM2_5_Mass_2020 = massDownloader('2020', zipped=True)
PM2_5_Mass_2021 = massDownloader('2021', zipped=True)

## Note about covid data
The covid 19 file is updated on a daily basis, if the file isn't reshaped or deleted this shouldn't be a problem, but in case it goes wrong, please download the dataset at this [google drive](https://drive.google.com/file/d/1Po0Uk1t_jk_5NGeW_MQAKyES_jv_t-zk/view?usp=sharing).

In [3]:
def covidMunger(df):
    """
    Takes the covid data set from NYT and returns
    a new dataframe with only the columns of interest.
    """
    # creates a copy of the dataframe without fips column 
    df = df.drop(columns=['fips']).copy()
    df['date'] = pd.to_datetime(df['date'])
    # The rows with NaN's are dropped
    df.dropna(inplace=True)
    return df


def pmmassMunger(df):
    """
    Takes the PM2.5 Mass from data set from aqs.epa.gov
    and returns a new dataframe with only the columns of
    interest.
    """
    # Select only the columns of interest
    df = df[['State Name', 'Date Local', 'AQI']].copy()
    # Set columns to correct data type
    df['Date Local'] = pd.to_datetime(df['Date Local'])
    # The rows with NaN's are dropped
    df.dropna(inplace=True)
    return df


covid19_usa = covidMunger(covid19_usa)
PM2_5_Mass_2020 = pmmassMunger(PM2_5_Mass_2020)
PM2_5_Mass_2021 = pmmassMunger(PM2_5_Mass_2021)

In [4]:
# Checking if the missing data was removed properly
print(covid19_usa.isnull().sum())
print(PM2_5_Mass_2020.isnull().sum())
print(PM2_5_Mass_2021.isnull().sum())

date      0
state     0
cases     0
deaths    0
dtype: int64
State Name    0
Date Local    0
AQI           0
dtype: int64
State Name    0
Date Local    0
AQI           0
dtype: int64


In [5]:
def covidYearSelect(df, begin, end):
    """
    Takes a dataframe containing covid data and
    a begin and an end year. It then returns a data frame
    containing only the selected period.
    """
    # From the covid dataset,only years 2020 and 2021 are necessary
    df_select = df[(df['date'].dt.year >= begin) & (df['date'].dt.year <= end)]
    return df_select


def massMerger(df1, df2):
    """
    Takes two dataframes containing PM2.5 Mass data and
    returned a merged file with renamed columns.
    """
    # Concatenates df1 and df2
    merged_mass = pd.concat([df1, df2], axis=0)
    # Renames the time columns to date for both dataframes
    merged_mass.rename(columns={'Date Local':'date','State Name':'state'},
                       inplace=True)
    return merged_mass


covid19_usa_20et21 = covidYearSelect(covid19_usa, 2020, 2021)
PM2_5_Mass_20et21 = massMerger(PM2_5_Mass_2020, PM2_5_Mass_2021)

In [6]:
# Shows if the dates have correctly been selected
print(f"Covid, Begin:{covid19_usa_20et21['date'].min()}, End:{covid19_usa_20et21['date'].max()}")
print(f"PM2.5 Mass 2020, Begin:{PM2_5_Mass_20et21['date'].min()}, End:{PM2_5_Mass_20et21['date'].max()}")

Covid, Begin:2020-01-21 00:00:00, End:2021-12-31 00:00:00
PM2.5 Mass 2020, Begin:2020-01-01 00:00:00, End:2021-11-10 00:00:00


Looking at the above plot, a few states immediately stand out, such as California, Texas, Florida and New York. Both of these have the largest number of covid cases and deaths. Considering however the number of states in the USA, this line plot might be useful for a simple glance, however the dataset needs to be further analysed to come to concrete conclusions.

The scatter plot for air quality gives close to no information at all. Nevada, Washington and California seem to stand out a little more then the rest.

In [7]:
def monthCalc_covid(df):
    """
    Takes a dataframe containing daily covid cases and deaths and
    returns the averages per month per state as a dataframe.
    """
    states = df['state'].unique()
    df_final = pd.DataFrame(columns=df.columns)

    for state in states:
        df_state = df[df['state'] == state]
        month_avg = df_state.groupby(pd.PeriodIndex(
            df_state['date'],
            freq="M"))[['cases', 'deaths']].sum().reset_index()
        month_avg = month_avg.assign(state=state)
        df_final = df_final.append(month_avg).reset_index(drop=True)

    return df_final


def monthCalc_air(df):
    """
    Takes a dataframe containing PM2.5 Mass and returns the average AQI
    per month per state as a dataframe.
    """
    states = PM2_5_Mass_20et21['state'].unique()
    df_final = pd.DataFrame(columns=df.columns)

    for state in states:
        df_state = df[df['state'] == state]
        month_avg = df_state.groupby(pd.PeriodIndex(
            df_state['date'], freq="M"))['AQI'].mean().reset_index()
        month_avg = month_avg.assign(state=state)
        df_final = df_final.append(month_avg).reset_index(drop=True)

    return df_final


monthlyCovid = monthCalc_covid(covid19_usa_20et21)
monthlyAir = monthCalc_air(PM2_5_Mass_20et21)

In [8]:
def converTime(df):
    """ 
    Converts the 'date' column from a data frame from dateperiod
    format to datetime. This format cannot be converted to datetime at once,
    however converting it to a string first made that possible.
    """
    df['date'] = df['date'].astype(str).copy()
    df['date'] = pd.to_datetime(df['date'])
    return df


monthlyCovid = converTime(monthlyCovid)
monthlyAir = converTime(monthlyAir)

In [9]:
def covidAirMerger(air_df, covid_df):
    """
    Takes two dataframes with one containing Air data and another
    one containing covid data returned a merged version.
    """
    # Creates a variable with the merged Covid and Air datasets
    merged = pd.merge(air_df, covid_df,
                        how='left', left_on=['state', 'date'],
                        right_on=['state', 'date'])
    
    # Removes any NANs that might have appeared
    merged = merged[~merged.isna().any(axis=1)]
    # Converts columns into the correct datatype as cases and
    merged[['AQI', 'cases', 'deaths']] = merged[[
        'AQI', 'cases', 'deaths']].apply(pd.to_numeric, errors='coerce')
    merged = merged.rename(columns={'cases':'covid cases', 'deaths':'covid deaths'})
    
    return merged


CA_Merge = covidAirMerger(monthlyAir, monthlyCovid)

In [10]:
# Observe whether the file columns types are correct
CA_Merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 950 entries, 2 to 1060
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   state         950 non-null    object        
 1   date          950 non-null    datetime64[ns]
 2   AQI           950 non-null    float64       
 3   covid cases   950 non-null    int64         
 4   covid deaths  950 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 44.5+ KB


In [11]:
# Creates a plot with covid cases and deaths for visual inspection
def quickliner(query, plottype):
    """ Takes 2 inputs, a dataframe containing covid data,
    and a query for plotting, which can be 'cases' or 'deaths'.
    Returns a figure with the desired output"""
    df = CA_Merge    
    if plottype == 'line':
        fig = px.line(df, x="date", y=query,
                      title=f'{query} in the USA', color='state')
    elif plottype == 'scatter':
        fig = px.scatter(df, x="date", y=query,
                      title=f'{query} in the USA', color='state')

    return fig


# Creates lists for the widget options
plottypes=['line', 'scatter']
query = ['covid cases', 'covid deaths', 'AQI']

# Creates the selection options for the widgets
select1 = pn.widgets.Select(name='Query', options=query)
select2 = pn.widgets.Select(name='Plot type', options=plottypes)
# Calls the widget
interact(quickliner, query=select1, plottype=select2)

Column
    [0] Column
        [0] Select(name='Query', options=['covid cases', ...], value='covid cases')
        [1] Select(name='Plot type', options=['line', 'scatter'], value='line')
    [1] Row
        [0] Plotly(Figure, name='interactive00103')

In [12]:
def CorrelScatter(query):
    """
    Takes a query and returns a scatter plot of the latter with
    with AQI values.
    """
    df = CA_Merge
    # Small constant epsilon avoids division by zero during regression
    epsilon = 1e-5
    # Creates a column with the log values of query
    df[f'log {query}'] = np.log(df[query]+epsilon)
    # Creates scatter plot with regression line
    fig = px.scatter(df, x=f'log {query}', y='AQI',
                     title=f'Correlation between {query} and AQI',
                     trendline='ols')
    # Sets the xaxis range from 0 to maximum value + 1
    fig.update_layout(xaxis_range=[-1,df[f'log {query}'].max(axis=0)+1])
    
    return fig


p1 = CorrelScatter('covid cases')
p2 = CorrelScatter('covid deaths')
tabs = pn.Tabs(('Covid cases', p1),('Covid deaths', p2))
tabs

Tabs
    [0] Plotly(Figure, name='Covid cases')
    [1] Plotly(Figure, name='Covid deaths')

In [13]:
def statePlotter(state, query):
    """
    Takes 2 input variables, state and query.
    The state can be any of the american states and
    query can be 'covid cases' or 'covid deaths'.
    Returns a plot containg the desired output.
    """
    df = CA_Merge
    if state == 'Show all':
        pass
    else:
        df = df[df['state'] == state]
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df['date'],
                   y=df[query],
                   name=query),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df['date'],
                   y=df['AQI'],
                   name="AQI score"),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title_text=f"Daily {query} and AQI scores in {state}"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="<b>date<b>")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Covid cases</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>AQI score</b>", secondary_y=True)

    return fig


state_list = list(CA_Merge['state'].unique())
state_list += ['Show all']
query = ['covid cases', 'covid deaths']

select1 = pn.widgets.Select(name='State', options=state_list)
select2 = pn.widgets.Select(name='Query', options=query)
interact(statePlotter, state=select1, query=select2)

Column
    [0] Column
        [0] Select(name='State', options=['Alabama', 'Alaska', ...], value='Alabama')
        [1] Select(name='Query', options=['covid cases', ...], value='covid cases')
    [1] Row
        [0] Plotly(Figure, name='interactive00121')

### Choice for number of groups

The lower number of groups (3 groups) did not seem to show a significant difference in between groups. However, due to each group containing a large quantity of information, this might be causing noise to hide any difference. For this reason each category was also split in "lower" and "upper" groups making a total of 5 categories.

In [14]:
# Creates a larger number of categories, as the large categories
# might be causing noise and obfusifying and differences.
category = pd.cut(CA_Merge.AQI,
                  bins=[0, 25, 50, 75, 100, 125],
                  labels=['Good_l', 'Good_u', 'Moderate_l',
                          'Moderate_u', 'Unhealthy_l'])
CA_Merge.insert(3, 'AQI_Category', category)

In [15]:
def ANOVA_summarizer(df, group):
    print('\n', f'ANOVA summery for {group} against AQI groups')
    print(rp.summary_cont(df[group].groupby(df['AQI_Category'])))
    # One-way ANOVA for relation between number of specified group and air quality
    print('\n', stats.f_oneway(df[group][df['AQI_Category'] == 'Good_l'],
                               df[group][df['AQI_Category'] == 'Good_u'],
                               df[group][df['AQI_Category'] == 'Moderate_l'],
                               df[group][df['AQI_Category'] == 'Moderate_u'],
                               df[group][df['AQI_Category'] == 'Unhealthy_l']))


ANOVA_summarizer(CA_Merge, 'covid cases')
ANOVA_summarizer(CA_Merge, 'covid deaths')


 ANOVA summery for covid cases against AQI groups


                N          Mean            SD            SE     95% Conf.  \
AQI_Category                                                                
Good_l        271  2.923023e+06  6.194324e+06  3.762784e+05  2.182210e+06   
Good_u        654  1.368619e+07  2.068392e+07  8.088054e+05  1.209802e+07   
Moderate_l     21  1.522859e+07  2.859031e+07  6.238918e+06  2.214438e+06   
Moderate_u      2  1.284768e+07  1.458080e+07  1.031018e+07 -1.181556e+08   
Unhealthy_l     2  6.243694e+06  7.557018e+06  5.343618e+06 -6.165342e+07   

                  Interval  
AQI_Category                
Good_l        3.663836e+06  
Good_u        1.527436e+07  
Moderate_l    2.824275e+07  
Moderate_u    1.438510e+08  
Unhealthy_l   7.414081e+07  

 F_onewayResult(statistic=17.504075587826478, pvalue=7.232376465429688e-14)

 ANOVA summery for covid deaths against AQI groups


                N         Mean           SD           SE     95% Conf.  \

### Interpretation ANOVA

According to the performed ANOVA, the different air quality groups seems to have show a significant difference, whereas covid cases returns a p-value=7.23e-14 and covid deaths returns p-value=1.38e-14. Considering these are both clearly a smaller then 0.05 it is very likely that these groups are significantly different from each other.

In [16]:
def violinist(query):
    """ Takes a query as input and used a dataframe containing
    AQI and covid information to create a violin plot.
    The query can be 'covid cases' or 'covid deaths'. """
    df = CA_Merge
    fig = go.Figure()

    AQI_Cats = ['Good_l', 'Good_u', 'Moderate_l', 'Moderate_u', 'Unhealthy_l']
    AQI_Cats_dict = {'Good_l':'Good lower', 'Good_u':'Good upper',
                     'Moderate_l':'Moderate lower', 'Moderate_u':'Moderate upper',
                     'Unhealthy_l':'Unhealthy lower'}

    for cat in AQI_Cats:
        fig.add_trace(go.Violin(x=df['AQI_Category'][df['AQI_Category'] == cat],
                                y=df[query][df['AQI_Category'] == cat],
                                name=AQI_Cats_dict[cat],
                                box_visible=True,
                                meanline_visible=True))
    
    fig.update_traces(meanline_visible=True,
                  points='all', # show all points
                  jitter=0.05,  # jitter makes points better visible
                  scalemode='count') #scales violin plot accorinding to total count
    fig.update_layout(
    title_text=f" <b> Number of {query} per air quality category <b>",
    violingap=0.3, violingroupgap=0.4, violinmode='overlay')

    return fig


p1 = violinist('covid cases')
p2 = violinist('covid deaths')
tabs = pn.Tabs(('Covid cases', p1),('Covid deaths', p2))
tabs

Tabs
    [0] Plotly(Figure, name='Covid cases')
    [1] Plotly(Figure, name='Covid deaths')

In [17]:
def theeCompare(df, query, cat1, cat2):
    tStat, pValue = stats.ttest_ind(df[query][df['AQI_Category'] == cat1],
                                    df[query][df['AQI_Category'] == cat2],
                                    equal_var = False)
    
    AQI_Cats_dict = {'Good_l':'Good lower', 'Good_u':'Good upper',
                     'Moderate_l':'Moderate lower', 'Moderate_u':'Moderate upper',
                     'Unhealthy_l':'Unhealthy lower'}

    print(f"{query}: {AQI_Cats_dict[cat1]} vs {AQI_Cats_dict[cat2]}")
    if pValue <= 0.05:
        print("Significant: Value={:.2E}".format(pValue), '\n')
    else:
        print("Not significant: Value={:.2E}".format(pValue), '\n')


theeCompare(CA_Merge, 'covid deaths', 'Good_u', 'Moderate_l')
theeCompare(CA_Merge, 'covid cases', 'Good_u', 'Moderate_l')
theeCompare(CA_Merge, 'covid deaths', 'Good_l', 'Good_u')
theeCompare(CA_Merge, 'covid cases', 'Good_l', 'Good_u')

covid deaths: Good upper vs Moderate lower
Not significant: Value=9.30E-01 

covid cases: Good upper vs Moderate lower
Not significant: Value=8.09E-01 

covid deaths: Good lower vs Good upper
Significant: Value=4.89E-29 

covid cases: Good lower vs Good upper
Significant: Value=4.23E-31 



### Violin plot and t-tests
By creating violin plots with jitters next to them, it became clear groups indeed seem to be different. Unhealthy lower and moderate upper, did seem t have very few data points and should perhaps not be included in the analysis. Out of the three remaining groups, no significant difference was found in between “good upper” (AQI 26-50) and “moderate lower” (AQI 51-75). The “lower good” (AQI 1-25) seemed to have far fewer covid cases or deaths, which was also confirmed by a t-test independent sample t-test.

## Conclusion
Finally, there does seem to be a difference in number covid cases/deaths in regards to AQI. Locations with a very low AQI, which indicates very good air quality, seems to show a far smaller number of cases. Considering [previous research](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2021GH000383), air quality indeed seems to have a correlation with covid propagation and mortality. 



Ambient air pollution exposure can already cause several respiratory illnesses, additionally some virusus can spread through fine air particles.